# 📊 Spreadsheet RAG with SpreadsheetChunker

This notebook demonstrates how to use the **SpreadsheetChunker** to process Excel files and create chunks suitable for RAG (Retrieval-Augmented Generation).

## What You'll Learn

- Load and chunk local Excel files
- Two chunking strategies:
  - **Row-by-row chunking**: Creates individual chunks for each data row
  - **Sheet-level chunking**: Creates one chunk per worksheet with AI-generated summaries
- Prepare spreadsheet data for embedding and vector search

> **Note**: This builds on the concepts from [3-basic-rag.ipynb](3-basic-rag.ipynb)

## 1. Setup and Installation

First, let's install the required packages for working with Excel files and token estimation.

In [ ]:
%pip install openpyxl tiktoken

## 2. Row-by-Row Chunking 📝

Let's demonstrate how to use the `SpreadsheetChunker` to process a local Excel file and create chunks suitable for RAG.

**What this does:**
- Loads a local Excel file from your filesystem
- Chunks the data by individual rows
- Optionally includes headers in each chunk
- Prepares data for embedding and indexing

**Note:** Make sure you have a sample Excel file ready. You can create one with sample data or use any existing `.xlsx` file.

In [ ]:
# Simple demo: Load a local Excel file and chunk it
from chunkers import SpreadsheetChunker
from pathlib import Path

# Replace with your actual Excel file path
excel_file_path = "docs/Weekly time record small business.xlsx"  # Change this to your file

# Helper function to create data dict for local files
def create_local_file_data(file_path: str):
    """Create data dictionary for SpreadsheetChunker from local file path."""
    file_path = Path(file_path).resolve()  # Convert to absolute path
    
    # Read file bytes
    with open(file_path, 'rb') as f:
        file_bytes = f.read()
    
    return {
        'documentUrl': file_path.as_uri(),  # file:// URL (needs absolute path)
        'documentSasToken': '',  # No SAS token for local files
        'fileName': file_path.name,
        'documentBytes': file_bytes,  # Actual file content
        'documentContent': ''  # Empty for Excel files
    }

# Create the data dictionary
data = create_local_file_data(excel_file_path)

# Create chunker with local file data
chunker = SpreadsheetChunker(
    data=data,
    max_chunk_size=1536,
    chunking_by_row=True,  # or False for sheet-based chunking
    include_header_in_chunks=True
)

# Get chunks
chunks = chunker.get_chunks()

# Display results
print(f"✅ Created {len(chunks)} chunks from {excel_file_path}")
for chunk in chunks[:10]:  # Show first 10 chunks
    if "summary" in chunk:
        print(chunk["summary"])
    print(f"\n📄 Chunk {chunk['chunk_id']}: {chunk['title']}")
    print(chunk['content'][:300] + "..." if len(chunk['content']) > 300 else chunk['content'])

## 3. Sheet-Level Chunking with AI Summaries 🤖

Instead of creating individual chunks for each row, let's try chunking **by sheet**. This approach:

- Creates one chunk per worksheet
- Uses AI to generate a summary of the entire sheet
- Better for high-level understanding of spreadsheet data
- Reduces total number of chunks for large spreadsheets

Set `chunking_by_row=False` to enable this mode.

In [ ]:
# Create chunker with sheet-level chunking (with AI summaries)
chunker_by_sheet = SpreadsheetChunker(
    data=data,
    max_chunk_size=1536,
    chunking_by_row=False,  # Chunk by sheet instead of by row
    include_header_in_chunks=False  # Not applicable for sheet-level chunking
)

# Get chunks
sheet_chunks = chunker_by_sheet.get_chunks()

# Display results
print(f"✅ Created {len(sheet_chunks)} chunks (one per sheet) from {excel_file_path}")
for chunk in sheet_chunks:
    print(f"\n📄 Chunk {chunk['chunk_id']}: {chunk['title']}")
    print(f"\n🤖 AI Summary:\n{chunk.get('summary', 'No summary available')}")
    print(f"\n📊 Table Data (first 500 chars):\n{chunk['content'][:500]}...")

## 4. Comparing Chunking Strategies

Let's compare the two approaches:

In [ ]:
print("📊 Chunking Strategy Comparison:")
print("=" * 60)
print(f"Row-by-row chunking: {len(chunks)} chunks")
print(f"Sheet-level chunking: {len(sheet_chunks)} chunks")
print("=" * 60)
print("\n💡 Use Cases:")
print("\nRow-by-row:")
print("  ✅ Granular search across individual records")
print("  ✅ When each row represents a distinct entity")
print("  ✅ Better for precise retrieval of specific data points")
print("\nSheet-level:")
print("  ✅ High-level understanding of data structure")
print("  ✅ When you need summaries of entire datasets")
print("  ✅ Reduces number of chunks for large spreadsheets")
print("  ✅ Better for understanding overall trends and patterns")

## 5. Next Steps: Integrating with RAG

Now that you have chunks, you can:

1. **Generate embeddings** for each chunk using the embedding model
2. **Upload to Azure AI Search** with vector search capability
3. **Query the index** to retrieve relevant spreadsheet data
4. **Use with chat model** to answer questions about your spreadsheet data

See [3-basic-rag.ipynb](3-basic-rag.ipynb) for the complete RAG pipeline implementation!

## 6. Conclusion

You've learned how to:

✅ Load and process local Excel files  
✅ Use row-by-row chunking for granular data access  
✅ Use sheet-level chunking with AI summaries  
✅ Prepare spreadsheet data for RAG pipelines  

Choose the chunking strategy that best fits your use case! 🎯